In [ ]:
import torch
torch.cuda.empty_cache()

## Make the Model

In [ ]:
data_path = 'attachment/attach_processed_length10.csv'
embeddings_path = 'attachment/doc/models/attach_doc_embeddings.npy'
model_save_path = "attachment/doc/models/attach_doc1_model.pkl"
data_save_path = "attachment/doc/models/attach_doc1_data.pkl"

data_path = 'attachment/attach_short.csv'
embeddings_path = 'attachment/doc/models/attach_short_embeddings.npy'
model_save_path = "attachment/doc/models/attach_short1_model.pkl"
data_save_path = "attachment/doc/models/attach_short1_data.pkl"

In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd

# Define file paths
data_path = 'rpg/rpg_small_processed_part1.csv'
embeddings_path = 'rpg/small/models/rpg_small_embeddings_part1.npy'

# Load the data
df = pd.read_csv(data_path, usecols=['text'], low_memory=False)

# Specify what the 'docs' are
docs = df['text'].tolist()

# Create embeddings
model = SentenceTransformer('thenlper/gte-large')
embeddings = model.encode(docs, show_progress_bar=True)

# Save Embeddings
with open(embeddings_path, 'wb') as f:
    np.save(f, embeddings)

c:\Users\snake\miniconda3\envs\maybert\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|██████████| 33267/33267 [1:32:25<00:00,  6.00it/s]  


In [2]:
import torch
torch.cuda.empty_cache()

In [3]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd

# Define file paths
data_path = 'rpg/rpg_small_processed_part2.csv'
embeddings_path = 'rpg/small/models/rpg_small_embeddings_part2.npy'

# Load the data
df = pd.read_csv(data_path, usecols=['text'], low_memory=False)

# Specify what the 'docs' are
docs = df['text'].tolist()

# Create embeddings
model = SentenceTransformer('thenlper/gte-large')
embeddings = model.encode(docs, show_progress_bar=True)

# Save Embeddings
with open(embeddings_path, 'wb') as f:
    np.save(f, embeddings)

Batches: 100%|██████████| 33267/33267 [1:28:31<00:00,  6.26it/s]  


In [4]:
import torch
torch.cuda.empty_cache()

RESTART HERE THIS IS WHERE I'VE STOPPED

In [1]:
from umap import UMAP
import datamapplot
from hdbscan import HDBSCAN
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import collections
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle as pickle
from bertopic.representation import MaximalMarginalRelevance
from bertopic.representation import KeyBERTInspired

data_path = 'rpg/rpg_small_processed_part1.csv'
embeddings_path = 'rpg/small/models/rpg_small_embeddings_part1.npy'
model_save_path = 'rpg/small/models/rpg_small_model_part1.pkl'
data_save_path = 'rpg/small/models/rpg_small_data_part1.pkl'

df = pd.read_csv(data_path, usecols=['text'], low_memory=False)
docs = df['text'].tolist()

embedding_model = SentenceTransformer('thenlper/gte-large')
embeddings = np.load(embeddings_path)

vocab = collections.Counter()
tokenizer = CountVectorizer(ngram_range=(1, 2)).build_tokenizer()
for doc in tqdm(docs):
    vocab.update(tokenizer(doc))
vocab = [word for word, frequency in vocab.items() if frequency >= 500]; len(vocab)

umap_model = UMAP(
        n_components=2,  # has a wild impact hard to predict
        n_neighbors=30,  # Higher is a more gloabl strcture
        min_dist=0.01,   # Lower value means more dense packing
        random_state=42, # Reproducability
        metric="cosine", # have to pick something
        n_jobs=-1        # speed
        )

hdbscan_model = HDBSCAN(
            min_cluster_size=500,               # smallest size group considered
            min_samples=200,                     # larger is more conservative - more noise
            leaf_size=75,                     # number of points per leaf node in the tree - default 40
            gen_min_span_tree=True,             # True creates minimum spanning trees - increasing RAM
            prediction_data=True,               # generates extra cached data of prediction labels for new data or reuse
            cluster_selection_method='leaf',  # eom is normal - leaf might get more homogeneous clusters
            # cluster_selection_epsilon=0.0,      # default - merges clusters below threshold
            core_dist_n_jobs=-1,                # For speed
            )

# Set the main_representation for the model
main_representation = KeyBERTInspired()
aspect_model = [KeyBERTInspired(top_n_words=10), MaximalMarginalRelevance(diversity=.2)]
representation_model = {
    "Main": main_representation,
    "Aspect1": aspect_model,
}

topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    representation_model=aspect_model,
    calculate_probabilities=True,
    verbose=True
    )

topics, probs = topic_model.fit_transform(docs, embeddings)

# Load the full dataset into the dataframe prior to loading
full_data = pd.read_csv(data_path, low_memory=False)
full_data['topics'] = topics

# Save the BERTopic model and the full dataset with topics to .pkl files
with open(model_save_path, "wb") as model_file:
    pickle.dump(topic_model, model_file)

with open(data_save_path, "wb") as data_file:
    pickle.dump(full_data, data_file)
   
print("Data and model saved successfully.")


c:\Users\snake\miniconda3\envs\maybert\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 1064539/1064539 [00:25<00:00, 42211.80it/s]
2024-05-15 11:33:21,643 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-05-15 12:02:49,405 - BERTopic - Dimensionality - Completed ✓
2024-05-15 12:02:49,412 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-05-15 12:37:35,129 - BERTopic - Cluster - Completed ✓
2024-05-15 12:37:35,229 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-05-15 12:46:34,987 - BERTopic - Representation - Completed ✓


Data and model saved successfully.


In [4]:
from bertopic import BERTopic

# Extract the results
topics = topic_model.get_topics()
topic_freq = topic_model.get_topic_freq()
topic_info = topic_model.get_topic_info()
representative_docs = topic_model.get_representative_docs()

# Save the results in a more structured and readable manner
csv_file_path = 'rpg/small/analysis/small_analysis_part1.csv'  # Hardcoded save location for CSV

with open('rpg/small/analysis/small_analysis_part1.txt', 'w') as f:
    # Topics
    f.write("TOPICS:\n")
    for topic_num, terms in topics.items():
        terms_str = ', '.join([term[0] for term in terms])
        f.write(f"Topic {topic_num}: {terms_str}\n")
    f.write("\n")
    
    # Topic Frequency
    f.write("TOPIC FREQUENCY:\n")
    for index, row in topic_freq.iterrows():
        f.write(f"Topic {row['Topic']}: {row['Count']} entries\n")
    f.write("\n")

    # Topic Info
    f.write("TOPIC INFO:\n")
    for index, row in topic_info.iterrows():
        f.write(f"Topic {row['Topic']}\n")
        f.write(f"  - Name: {row['Name']}\n")
        f.write("  - Representation:\n")
        for term in row['Representation']:
            f.write(f"    * {term}\n")
        f.write("\n")
    
    # Representative Docs
    f.write("REPRESENTATIVE DOCS:\n")
    for topic_num, docs in representative_docs.items():
        f.write(f"Topic {topic_num} representative docs:\n")
        for doc in docs:
            f.write(f"  - {doc}\n")
        f.write("\n")
        
import pandas as pd

# Assuming 'topic_info' is already defined and includes topic representations

# Convert 'topic_info' DataFrame directly to CSV
topic_info.to_csv(csv_file_path, index=False)

In [5]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,665440,-1_dm_character_play_game,"[dm, character, play, game, party, campaign, p...","[OnlineDiscord5E, any that fits, or Freeform C..."
1,0,16192,0_playing_character_play_dm,"[playing, character, play, dm, session, game, ...",[My DnD Group Of 2 Years Is Falling Apart And ...
2,1,11859,1_backstories_backstory_character_characters,"[backstories, backstory, character, characters...","[Personally, I'd say don't. If a player is hav..."
3,2,9128,2_spells_spellbook_spellcasting_spell,"[spells, spellbook, spellcasting, spell, sorce...","[spells are magic. spell damage is magic, Spel..."
4,3,8153,3_someone_friends_friend_situation,"[someone, friends, friend, situation, mental, ...",[Having strangers tell you how to feel can be ...
...,...,...,...,...,...
231,230,513,230_potions_potion_drinking_drink,"[potions, potion, drinking, drink, healing, he...",[I use a couple pretty basic home brew rules f...
232,231,513,231_attack_combat_attacks_disengage,"[attack, combat, attacks, disengage, action, d...",[If your enemies can't see you or you have no ...
233,232,510,232_session_zero_discussion_discuss,"[session, zero, discussion, discuss, sessions,...",[The key element of a good Session Zero discus...
234,233,507,233_hydra_hydras_false_not,"[hydra, hydras, false, not, creature, monsters...","[If you want a false hydra, this is how to do ..."


In [1]:
from umap import UMAP
import datamapplot
from hdbscan import HDBSCAN
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import collections
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle as pickle
from bertopic.representation import MaximalMarginalRelevance
from bertopic.representation import KeyBERTInspired

data_path = 'rpg/rpg_small_processed_part2.csv'
embeddings_path = 'rpg/small/models/rpg_small_embeddings_part2.npy'
model_save_path = 'rpg/small/models/rpg_small_model_part2.pkl'
data_save_path = 'rpg/small/models/rpg_small_data_part1.pkl'

df = pd.read_csv(data_path, usecols=['text'], low_memory=False)
docs = df['text'].tolist()

embedding_model = SentenceTransformer('thenlper/gte-large')
embeddings = np.load(embeddings_path)

vocab = collections.Counter()
tokenizer = CountVectorizer(ngram_range=(1, 3)).build_tokenizer()
for doc in tqdm(docs):
    vocab.update(tokenizer(doc))
vocab = [word for word, frequency in vocab.items() if frequency >= 100]; len(vocab)

umap_model = UMAP(
        n_components=3,  # has a wild impact hard to predict
        n_neighbors=30,  # Higher is a more gloabl strcture
        min_dist=0.01,   # Lower value means more dense packing
        random_state=42, # Reproducability
        metric="cosine", # have to pick something
        n_jobs=-1        # speed
        )

hdbscan_model = HDBSCAN(
            min_cluster_size=100,               # smallest size group considered
            min_samples=100,                     # larger is more conservative - more noise
            # leaf_size=40,                     # number of points per leaf node in the tree - default 40
            gen_min_span_tree=True,             # True creates minimum spanning trees - increasing RAM
            prediction_data=True,               # generates extra cached data of prediction labels for new data or reuse
            # cluster_selection_method='leaf',  # eom is normal - leaf might get more homogeneous clusters
            # cluster_selection_epsilon=0.0,      # default - merges clusters below threshold
            core_dist_n_jobs=-1,                # For speed
            )

# Set the main_representation for the model
main_representation = KeyBERTInspired()
aspect_model = [KeyBERTInspired(top_n_words=10), MaximalMarginalRelevance(diversity=.2)]
representation_model = {
    "Main": main_representation,
    "Aspect1": aspect_model,
}

topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    representation_model=aspect_model,
    calculate_probabilities=True,
    verbose=True
    )

topics, probs = topic_model.fit_transform(docs, embeddings)

# Load the full dataset into the dataframe prior to loading
full_data = pd.read_csv(data_path, low_memory=False)
full_data['topics'] = topics

# Save the BERTopic model and the full dataset with topics to .pkl files
with open(model_save_path, "wb") as model_file:
    pickle.dump(topic_model, model_file)

with open(data_save_path, "wb") as data_file:
    pickle.dump(full_data, data_file)
   
print("Data and model saved successfully.")


c:\Users\snake\miniconda3\envs\maybert\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 1064539/1064539 [00:24<00:00, 43014.89it/s]
2024-05-15 15:40:07,501 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-05-15 16:13:07,921 - BERTopic - Dimensionality - Completed ✓
2024-05-15 16:13:07,932 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-05-15 21:42:27,753 - BERTopic - Cluster - Completed ✓
2024-05-15 21:42:27,906 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-05-15 21:54:14,770 - BERTopic - Representation - Completed ✓


Data and model saved successfully.


In [2]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,623934,-1_dm_character_play_game,"[dm, character, play, game, games, them, playe...",[You Can't Buy an Imaginary Girlfriend For Fif...
1,0,29713,0_someone_yourself_say_you,"[someone, yourself, say, you, saying, guy, yes...","[isn't... OP... that guy here lol, Know that w..."
2,1,12832,1_dice_d20s_d20_rolls,"[dice, d20s, d20, rolls, roll, random, die, pr...","[I get the question, however the die size does..."
3,2,11859,2_roll_rolls_rolling_rolled,"[roll, rolls, rolling, rolled, dice, skill, si...","[They roll with it., DM interpretations and ro..."
4,3,9026,3_dnd_dming_dm_playing,"[dnd, dming, dm, playing, dms, behavior, game,...",[Advice on a borderline toxic playerdm Excuse ...
...,...,...,...,...,...
536,535,102,535_chat_talk_rrpg_free,"[chat, talk, rrpg, free, podcast, games, enjoy...",[Free chat - 2018-12-15 Come here and talk abo...
537,536,101,536_link_linked_linking_links,"[link, linked, linking, links, webpage, see, c...","[Here's the link., Link I think, Link for my p..."
538,537,101,537_timeloop_loops_loop_temporal,"[timeloop, loops, loop, temporal, time, parado...",[Tips and ideas for a time-loopGroundhog Day g...
539,538,101,538_golden_sky_rpg_game,"[golden, sky, rpg, game, games, play, miyazaki...","[Golden Sky Stories is a really sweet game., I..."


In [3]:
from bertopic import BERTopic

# Extract the results
topics = topic_model.get_topics()
topic_freq = topic_model.get_topic_freq()
topic_info = topic_model.get_topic_info()
representative_docs = topic_model.get_representative_docs()

# Save the results in a more structured and readable manner
csv_file_path = 'rpg/small/analysis/small_analysis_part2.csv'  # Hardcoded save location for CSV

with open('rpg/small/analysis/small_analysis_part2.txt', 'w') as f:
    # Topics
    f.write("TOPICS:\n")
    for topic_num, terms in topics.items():
        terms_str = ', '.join([term[0] for term in terms])
        f.write(f"Topic {topic_num}: {terms_str}\n")
    f.write("\n")
    
    # Topic Frequency
    f.write("TOPIC FREQUENCY:\n")
    for index, row in topic_freq.iterrows():
        f.write(f"Topic {row['Topic']}: {row['Count']} entries\n")
    f.write("\n")

    # Topic Info
    f.write("TOPIC INFO:\n")
    for index, row in topic_info.iterrows():
        f.write(f"Topic {row['Topic']}\n")
        f.write(f"  - Name: {row['Name']}\n")
        f.write("  - Representation:\n")
        for term in row['Representation']:
            f.write(f"    * {term}\n")
        f.write("\n")
    
    # Representative Docs
    f.write("REPRESENTATIVE DOCS:\n")
    for topic_num, docs in representative_docs.items():
        f.write(f"Topic {topic_num} representative docs:\n")
        for doc in docs:
            f.write(f"  - {doc}\n")
        f.write("\n")
        
# Convert 'topic_info' DataFrame directly to CSV
topic_info.to_csv(csv_file_path, index=False)

#### Analysis

In [ ]:
from bertopic import BERTopic

# Extract the results
topics = topic_model.get_topics()
topic_freq = topic_model.get_topic_freq()
topic_info = topic_model.get_topic_info()
representative_docs = topic_model.get_representative_docs()

# Save the results in a more structured and readable manner
csv_file_path = 'rpg/analysis/small_analysis_part2.csv'  # Hardcoded save location for CSV

with open('rpg/analysis/small_analysis_part2.txt', 'w') as f:
    # Topics
    f.write("TOPICS:\n")
    for topic_num, terms in topics.items():
        terms_str = ', '.join([term[0] for term in terms])
        f.write(f"Topic {topic_num}: {terms_str}\n")
    f.write("\n")
    
    # Topic Frequency
    f.write("TOPIC FREQUENCY:\n")
    for index, row in topic_freq.iterrows():
        f.write(f"Topic {row['Topic']}: {row['Count']} entries\n")
    f.write("\n")

    # Topic Info
    f.write("TOPIC INFO:\n")
    for index, row in topic_info.iterrows():
        f.write(f"Topic {row['Topic']}\n")
        f.write(f"  - Name: {row['Name']}\n")
        f.write("  - Representation:\n")
        for term in row['Representation']:
            f.write(f"    * {term}\n")
        f.write("\n")
    
    # Representative Docs
    f.write("REPRESENTATIVE DOCS:\n")
    for topic_num, docs in representative_docs.items():
        f.write(f"Topic {topic_num} representative docs:\n")
        for doc in docs:
            f.write(f"  - {doc}\n")
        f.write("\n")
        
import pandas as pd

# Assuming 'topic_info' is already defined and includes topic representations

# Convert 'topic_info' DataFrame directly to CSV
topic_info.to_csv(csv_file_path, index=False)

## Merge the Models

In [ ]:
# Import necessary libraries
from bertopic import BERTopic
import pickle

# Load the pre-trained BERTopic models
topic_model_1 = BERTopic.load('rpg/small/models/rpg_small_model_part1.pkl')
topic_model_2 = BERTopic.load('rpg/small/models/rpg_small_model_part2.pkl')

# Set the 'min_similarity' value for merging
min_similarity = 0.6  # Adjust this value as needed

# Merge the models
merged_model = BERTopic.merge_models([topic_model_1, topic_model_2], min_similarity=min_similarity)

# Save the merged model
with open('rpg/small/models/rpg_small_merged_model.pkl', 'wb') as file:
    pickle.dump(merged_model, file)

# Provide information on the number of topics
num_topics_first_model = topic_model_1.get_topic_info().shape[0]
num_topics_merged_model = merged_model.get_topic_info().shape[0]

# Calculate the difference in the number of topics
difference = num_topics_merged_model - num_topics_first_model

print(f"The first model had {num_topics_first_model} topics.")
print(f"The merged model has {num_topics_merged_model} topics.")
print(f"There is a difference of {difference} topics between the first and merged models.")


In [3]:
# Import necessary libraries
from bertopic import BERTopic
import pickle

# Load the pre-trained BERTopic models
topic_model_1 = BERTopic.load('rpg/small/models/rpg_small_model_part1.pkl')
topic_model_2 = BERTopic.load('rpg/small/models/rpg_small_model_part2.pkl')

# Provide information on the number of topics
num_topics_first_model = topic_model_1.get_topic_info().shape[0]
num_topics_merged_model = topic_model_2.get_topic_info().shape[0] #### RETURN TO MERGED TOPIC MODEL NOT ## 2 #####

# Calculate the difference in the number of topics
difference = num_topics_merged_model - num_topics_first_model

print(f"The first model had {num_topics_first_model} topics.")
print(f"The merged model has {num_topics_merged_model} topics.")
print(f"There is a difference of {difference} topics between the first and merged models.")

The first model had 236 topics.
The merged model has 541 topics.
There is a difference of 305 topics between the first and merged models.


In [1]:
# Import necessary libraries
from bertopic import BERTopic
import pickle

# Load the pre-trained BERTopic models
topic_model_1 = BERTopic.load('rpg/small/models/rpg_small_model_part1.pkl')
topic_model_2 = BERTopic.load('rpg/small/models/rpg_small_model_part2.pkl')

# Set the 'min_similarity' value for merging
min_similarity = 0.6  # Adjust this value as needed

# Merge the models
merged_model = BERTopic.merge_models([topic_model_1, topic_model_2], min_similarity=min_similarity)

# Save the merged model
with open('rpg/small/models/rpg_small_merged_model.pkl', 'wb') as file:
    pickle.dump(merged_model, file)

# Provide information on the number of topics
num_topics_first_model = len(topic_model_1.get_topic_info())
num_topics_merged_model = len(merged_model.get_topic_info())

# Calculate the difference in the number of topics
difference = num_topics_merged_model - num_topics_first_model

print(f"The first model had {num_topics_first_model} topics.")
print(f"The merged model has {num_topics_merged_model} topics.")
print(f"There is a difference of {difference} topics between the first and merged models.")


c:\Users\snake\miniconda3\envs\maybert\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The first model had 236 topics.
The merged model has 236 topics.
There is a difference of 0 topics between the first and merged models.


In [1]:
# Import necessary libraries
from bertopic import BERTopic
import pickle
from tqdm import tqdm

# Load the pre-trained BERTopic models
print("Loading pre-trained BERTopic models...")
topic_model_1 = BERTopic.load('rpg/small/models/rpg_small_model_part1.pkl')
topic_model_2 = BERTopic.load('rpg/small/models/rpg_small_model_part2.pkl')
print("Pre-trained BERTopic models loaded successfully.")

# Set the 'min_similarity' value for merging
min_similarity = 0.6 # Adjust this value as needed

# Merge the models
print("Merging BERTopic models...")
merged_model = BERTopic.merge_models([topic_model_1, topic_model_2], min_similarity=min_similarity)
print("BERTopic models merged successfully.")

# Save the merged model
print("Saving the merged model...")
with open('rpg/small/models/rpg_small_merged_model.pkl', 'wb') as file:
    pickle.dump(merged_model, file)
print("Merged model saved.")

# Provide information on the number of topics
num_topics_first_model = len(topic_model_1.get_topic_info())
num_topics_merged_model = len(merged_model.get_topic_info())

# Calculate the difference in the number of topics
difference = num_topics_merged_model - num_topics_first_model

print(f"The first model had {num_topics_first_model} topics.")
print(f"The merged model has {num_topics_merged_model} topics.")
print(f"There is a difference of {difference} topics between the first and merged models.")

# Simulate a progress bar
print("Simulating progress bar...")
for _ in tqdm(range(100), desc="Progress"):
    pass
print("Code execution completed."
      )

c:\Users\snake\miniconda3\envs\maybert\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading pre-trained BERTopic models...
Pre-trained BERTopic models loaded successfully.
Merging BERTopic models...
BERTopic models merged successfully.
Saving the merged model...
Merged model saved.
The first model had 236 topics.
The merged model has 236 topics.
There is a difference of 0 topics between the first and merged models.
Simulating progress bar...


Progress: 100%|██████████| 100/100 [00:00<?, ?it/s]

Code execution completed.


In [ ]:
# Import necessary libraries
from bertopic import BERTopic
import pickle
from tqdm import tqdm

# Load the pre-trained BERTopic models
print("Loading pre-trained BERTopic models...")
topic_model_1 = BERTopic.load('rpg/small/models/rpg_small_model_part1.pkl')
topic_model_2 = BERTopic.load('rpg/small/models/rpg_small_model_part2.pkl')
print("Pre-trained BERTopic models loaded successfully.")

## Revise and Visualize the model

In [7]:
from umap import UMAP
import datamapplot
from hdbscan import HDBSCAN
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import pandas as pd
import collections
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle as pickle


model_save_path = "attachment/doc/models/attach_doc2_model.pkl"
data_save_path = "attachment/doc/models/attach_doc2_data.pkl"
embeddings_path = 'attachment/doc/models/attach_doc_embeddings.npy'

embedding_model = SentenceTransformer('thenlper/gte-large')
embeddings = np.load(embeddings_path)

topic_model = BERTopic.load(model_save_path)

with open(data_save_path, "rb") as file:
    df = pickle.load(file)
    
docs = df['text'].tolist()

In [ ]:
topic_model.get_topic_info()

In [11]:
intertopic_distance_map = topic_model.visualize_topics(title='<b>Closeness of Topic Centroids</b>', width=1000, height=1000)
intertopic_distance_map.show()
intertopic_distance_map.write_html("attachment/doc/visualizations/intertopic_distance_map_doc2.html")

In [9]:
hierarchical_topics = topic_model.hierarchical_topics(docs)

# Print topic tree
tree = topic_model.get_topic_tree(hierarchical_topics)
print(tree)

100%|██████████| 173/173 [00:50<00:00,  3.41it/s]

.
├─relationship_love_being_da_someone
│    ├─relationship_feel_being_someone_need
│    │    ├─intimacy_libido_sex_sexually_sexuality
│    │    │    ├─touching_affection_touch_affectionate_love
│    │    │    │    ├─■──affection_love_affirmation_gifts_words ── Topic: 88
│    │    │    │    └─■──intimacy_touching_affection_touch_affectionate ── Topic: 113
│    │    │    └─intimacy_sex_sexual_sexually_sexuality
│    │    │         ├─intimacy_sex_libido_sexually_sexual
│    │    │         │    ├─■──asexuality_asexual_asexuals_demisexuality_demisexual ── Topic: 159
│    │    │         │    └─sex_intimacy_sexually_libido_sexual
│    │    │         │         ├─■──sex_intimacy_sexually_libido_sexual ── Topic: 28
│    │    │         │         └─■──intimacy_sex_libido_sexual_sexually ── Topic: 145
│    │    │         └─■──porn_masturbation_masturbating_sexual_relationship ── Topic: 123
│    │    └─relationship_da_being_someone_love
│    │         ├─relationship_after_feel_someone_fa
│    │     

In [22]:
# Visualize these representations
hierarchical_fig = topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

hierarchical_fig.write_html("attachment/doc/visualizations/document2_hierarchical_visualization.html")

In [ ]:
# Reduce dimensionality of embeddings, this step is optional
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.1, metric='cosine').fit_transform(embeddings)
fig = topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings, hide_annotations=True)
fig.write_html("attachment/doc/visualizations/document2_umap_visualization.html")

In [25]:
timestamps = df['created_utc'].tolist()

topics_over_time = topic_model.topics_over_time(docs, timestamps)
topic_model.visualize_topics_over_time(topics_over_time)